In [2]:
from __future__ import division
from scipy import linalg as la
from scipy import optimize
import sympy
import numpy as np
import matplotlib.pyplot as plt
sympy.init_printing()

In [3]:
"""
Non linear equations cannot be written as a matrix-vector multiplication
instead, a system of multivariate nonlinear equations are written as a vector valued functions

for example: f:Rn -> Rn that takes n dimensionlal vector and maps it to another 
n-dimensional vector

Newton's method can be used for multivariate problems, and in this case, its 
iteration formular id

xk+1 = xk - Jf(xk)`(f(xk)) where Jf(xk) is the Jacobian matrix of the function f(x)
with elements [Jf(xk)ij] = dfi(xk)/dxi

instead of inverting the jacobian matrix, it is sufficient to solve the 
linear equation system Jf(xk)dxk = -f(xk) and update xk using xk + dxk like the secant variables of the newton method
that avoid computing the jacobian by estimating it from previous function evaluations.

Broyden's method is a particular exapmle of this method for multivariate equation systems.
in the scipy optimize module, broyden 1 and broyden 2 provide two implementations
of Broyden's method using different approximations of the jacobian and the 
function optimize.fsolve that provides an implementation of Newton-like method
where optionally th jacobian can be specified if available

the functions all have similar function signatures
arguments:
-python function representing the equation to be solved, and it should take an array
and it should take a numpy array as the first argument and it should return an array of the same shape
-the secon argument is an inittial guess for the solution as a numpy array
-optimize.fsole function also takes fprime as an optional keyword argument
which can be used to provide a function that eturns the Jacobian of the function of fx
-In addititon, there are other arguments for fine tuning, see the docstrings

"""

# for example, consider the following system of 2 multivariate and nonlinear equations

# y - x**3 -2x**2 + 1 = 0
# y + x**2 -1 = 0

# this can be represented by the vector valued function

# f([x1, x2]) = [x2 - x**3 - 2x**2 + 1, x2 + x**2 - 1]

# to solve this using scipy we need to define python function for f([x1, x2])
# and call, for example the optimize.fsolve using the function and an inititial guess for the vector

def f(x):
    return [x[1] - x[0]**3 -2*x[0]**2 + 1, x[1] + x[0]**2 - 1]

optimize.fsolve(f, [1, 1])

array([0.73205081, 0.46410162])